# Proyecto 03 - Sistemas de Recomendación

## Dataset: STEAM

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Son dos archivos, uno de calificaciones y otro de información sobre los juegos.**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. Sin embargo, **deberás** modificarlas para hacer un mejor manejo de datos. Algunas cosas a las que deberás prestar atención (tal vez no a todas):
1. Tipos de datos: elige tipos de datos apropiados para cada columna.
2. Descartar columnas poco informativas.
3. Guardar en memoria datasets preprocesados para no tener que repetir código que tarde en correr.

### Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿cuáles son los juegos más populares?¿Y los menos populares?

### Filtro Colaborativo

Deberás implementar un sistema de recomendación colaborativo para este dataset. Ten en cuenta:

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.

Puedes implementar un filtro colaborativo a partir de la similitud coseno o índice de Jaccard. ¿Puedes utilizar los métodos de la librería Surprise? Si no es así, busca implementaciones (por ejemplo, nuevas librerías) que sean apropiadas.

Para comenzar a trabajar, puedes asumir que cada entrada es un enlace entre una persona usuaria y un item, **independientemente** de si la crítica es buena o mala. 

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Cómo harías para ponerle un valor a la calificación?
1. ¿Cómo harías para agregar contenido? Por ejemplo, cuentas con el género, precio, fecha de lanzamiento y más información de los juegos.
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

## Preprocesamiento

### Importamos las librerías

In [2]:
# Importamos las librerías que vamos a necesitar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from tqdm.notebook import tqdm
import gzip
import re
import nltk
from sklearn.preprocessing import MinMaxScaler
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV

### Configuraciones

In [3]:
# Le doy formato a los gráficos
# sns.set_style("darkgrid")
sns.set_style("white")
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('figure', titlesize=16)
plt.rc('axes', labelsize=14) 
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12) 

In [4]:
#Seteamos para que no utilice notacion cientifica
pd.options.display.float_format = '{:.4f}'.format
#Seteo para que el máximo de columnas que muestra al levantar una base sean 500
pd.set_option('display.max_columns',500)
#Estos códigos hacen que la visualización de la consola abarque toda la pantalla (sin los recortes a los costados). Tambien hacen que al mostrar dataframes podamos ver todas las columnas que tiene.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# Instanciamos método para ver el progreso de las operaciones sobre sobre los dataframes
tqdm.pandas()
# Indicador de tiempo de ejecución
%load_ext autotime

In [6]:
# Las siguientes lineas permiten habilitar o deshabilitar ciertas operaciones
# procesamiento de datos para ahorrar tiempo de procesamiento.
READ_RAW = True
PROCESS_TEXT = True
POLARITY = True

### Carga y limpieza inicial de datos

### Carga y limpieza inicial de datos

In [7]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

Dataset de **Reviews**

In [8]:
# Atencion: la carga de los datos tarda alrededor de 5 minutos 30 segundos
if READ_RAW:
    contador = 0
    data_reviews = []
    # Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n = 3, 
    # abrira uno de cada tres, y asi.
    n = 10
    for l in tqdm(parse('steam_reviews.json.gz')):
        if contador%n == 0:
            data_reviews.append(l)
        else:
            pass
        contador += 1
    data_reviews = pd.DataFrame(data_reviews)

0it [00:00, ?it/s]

In [9]:
if READ_RAW:
    data_reviews.to_csv('data_reviews.csv', index_label='index')
else:
    data_reviews = pd.read_csv('data_reviews.csv')
    data_reviews.drop(columns='index', inplace=True)

In [10]:
# Previsualizamos las primeras 5 instancias del dataset
data_reviews.head()

,username,hours,products,product_id,page_order,date,text,early_access,page,user_id,compensation,found_funny
0,Chaos Syren,0.1000,41.0000,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,Ariman1,13.2000,1386.0000,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,NaN,NaN
2,freakfantom,0.1000,1706.0000,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,NaN,NaN
3,The_Cpt_FROGGY,7.8000,2217.0000,631920,0,2017-12-11,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,False,1,76561198003030375,Product received for free,NaN
4,the_maker988,8.2000,18.0000,35140,7,2018-01-02,"This game was way to linear for me, and compar...",False,1,76561198353396844,NaN,NaN


In [11]:
# Previsualizamos las ultimas 5 instancias del dataset
data_reviews.tail()

,username,hours,products,product_id,page_order,date,text,early_access,page,user_id,compensation,found_funny
779302,Vidaar,783.5000,353.0000,252490,1,2013-12-11,Summary: Rust is a survival game created by Fa...,True,10217,NaN,NaN,NaN
779303,Nikolai Belinski,55.1000,204.0000,252490,1,2013-12-11,Coolest Game Eva,True,10218,NaN,NaN,NaN
779304,RancorZealot,203.5000,173.0000,252490,1,2013-12-11,Should you buy RUST? Uh... YES. The BEST Survi...,True,10219,76561197963920623,NaN,NaN
779305,Jerry,139.8000,79.0000,252490,1,2013-12-11,Great game!,True,10220,76561198042664856,NaN,NaN
779306,Helk,15375.0000,98.0000,252490,1,2013-12-11,I love this game!,True,10221,NaN,NaN,36.0000


In [12]:
# Reviso la informacion general del dataset
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779307 entries, 0 to 779306
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   username      779307 non-null  object 
 1   hours         776670 non-null  float64
 2   products      777741 non-null  float64
 3   product_id    779307 non-null  object 
 4   page_order    779307 non-null  int64  
 5   date          779307 non-null  object 
 6   text          779307 non-null  object 
 7   early_access  779307 non-null  bool   
 8   page          779307 non-null  int64  
 9   user_id       317340 non-null  object 
 10  compensation  14588 non-null   object 
 11  found_funny   120164 non-null  float64
dtypes: bool(1), float64(3), int64(2), object(6)
memory usage: 66.1+ MB


In [13]:
# Vemos los missings
data_reviews.isna().sum()

username             0
hours             2637
products          1566
product_id           0
page_order           0
date                 0
text                 0
early_access         0
page                 0
user_id         461967
compensation    764719
found_funny     659143
dtype: int64

In [14]:
# Selecciono las columnas que me interesan y que creo que pueden ser de utilidad
columnas = ['username', 'hours', 'product_id', 'text','user_id']
data_reviews = data_reviews[columnas]
data_reviews.head()

,username,hours,product_id,text,user_id
0,Chaos Syren,0.1000,725280,This would not be acceptable as an entertainme...,NaN
1,Ariman1,13.2000,328100,Addictive RPG ! Works fine on linux though it ...,NaN
2,freakfantom,0.1000,725280,Прикольная стрелялка. Взял дешево на распродаже.,NaN
3,The_Cpt_FROGGY,7.8000,631920,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,76561198003030375
4,the_maker988,8.2000,35140,"This game was way to linear for me, and compar...",76561198353396844


In [15]:
# Vemos los missings
data_reviews.isna().sum()

username           0
hours           2637
product_id         0
text               0
user_id       461967
dtype: int64

In [16]:
# Para las instancias que no tienen 'user id' voy a generar uno sintetico
# Como la columna de 'user id' tiene muchos missings, voy a generar un id
# para cada 'username'
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data_reviews['user_id'] = le.fit_transform(data_reviews.username)

# data_reviews1 = data_reviews[~data_reviews.user_id.isna()]
# data_reviews2 = data_reviews[data_reviews.user_id.isna()]
# data_reviews1['uid'] = data_reviews1['user_id']
# data_reviews2['uid'] = le.fit_transform(data_reviews2.username)
# data_reviews = pd.concat([data_reviews1, data_reviews2])
data_reviews.head()

,username,hours,product_id,text,user_id
0,Chaos Syren,0.1000,725280,This would not be acceptable as an entertainme...,62543
1,Ariman1,13.2000,328100,Addictive RPG ! Works fine on linux though it ...,26902
2,freakfantom,0.1000,725280,Прикольная стрелялка. Взял дешево на распродаже.,434604
3,The_Cpt_FROGGY,7.8000,631920,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,347904
4,the_maker988,8.2000,35140,"This game was way to linear for me, and compar...",503851


In [17]:
# Chqueo que no tengo missings
data_reviews.isna().sum()

username         0
hours         2637
product_id       0
text             0
user_id          0
dtype: int64

In [18]:
# Verifico los tipos de dato
data_reviews.dtypes

username       object
hours         float64
product_id     object
text           object
user_id         int32
dtype: object

In [19]:
# Redefino el tipo de dato de 'user_id' a int64
data_reviews['product_id'] = data_reviews['product_id'].astype(dtype='int64')
data_reviews.dtypes

username       object
hours         float64
product_id      int64
text           object
user_id         int32
dtype: object

In [20]:
# Exploro los estadisticos del dataset
data_reviews.describe(include='object')

,username,text
count,779307,779307
unique,540341,718779
top,123,
freq,213,1837


Dataset de **Games**

In [21]:
data_games = []
for l in parse('steam_games.json.gz'):
    data_games.append(l)
data_games = pd.DataFrame(data_games)

In [22]:
# Previsualizo las primeras 5 instancias
data_games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.4900,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.9900,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.8300,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.9900,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.7900,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.9900,False,773570,NaN,NaN,NaN


In [23]:
# Previsualizo las ultimas 5 instancias
data_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.4900,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.9900,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.2400,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.9900,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.3900,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.9900,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.9900,False,658870,"xropi,stev3ns",1 user reviews,NaN
32134,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",NaN,http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.9900,True,681550,NaN,Positive,NaN


In [24]:
# Reviso la informacion general del dataset
data_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [25]:
# Vemos los missings
data_games.isna().sum()

publisher          8052
genres             3283
app_name              2
title              2050
url                   0
release_date       2067
tags                163
discount_price    31910
reviews_url           2
specs               670
price              1377
early_access          0
id                    2
developer          3299
sentiment          7182
metascore         29458
dtype: int64

In [26]:
# Selecciono las columnas que me interesan
# Descarto la columna 'title' ya que tienen muchos missings y la columna 'app_name' tiene los nombres de los juegos
columnas = ['publisher', 'genres', 'app_name', 'tags', 'specs', 'price', 'id', 'developer', 'sentiment']
data_games = data_games[columnas]
data_games.head()

,publisher,genres,app_name,tags,specs,price,id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.9900,761140,Kotoshiro,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",[Single-player],0.9900,767400,彼岸领域,NaN
4,NaN,NaN,Log Challenge,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.9900,773570,NaN,NaN


In [27]:
# Trabajaremos sobre una copia del dataset de reviews
df_reviews = data_reviews.copy()

In [28]:
# Vemos los missings
data_games.isna().sum()

publisher    8052
genres       3283
app_name        2
tags          163
specs         670
price        1377
id              2
developer    3299
sentiment    7182
dtype: int64

In [29]:
# Descarto los missings de 'id' y de 'app_name'
data_games.dropna(axis=0, subset=['id', 'app_name'], inplace=True)

In [30]:
# Observamos los tipos de dato
data_games.dtypes

publisher    object
genres       object
app_name     object
tags         object
specs        object
price        object
id           object
developer    object
sentiment    object
dtype: object

In [31]:
# Cambio el tipo de dato de la columna 'id' para que coincida con la del dataframe 'data_reviews' 
data_games['id'] = data_games['id'].astype('int64')

In [32]:
# Chequeo los data types
data_games.dtypes

publisher    object
genres       object
app_name     object
tags         object
specs        object
price        object
id            int64
developer    object
sentiment    object
dtype: object

### Procesamiento de texto

A continuacion vamos a procesar el campo 'text' (que contiene los comentarios de las reviews) para inferir un 'rating' y ver si ese juego es de la preferencia o no del usuario.

In [33]:
# Trabajaremos sobre una copia del dataset de reviews
df_reviews = data_reviews.copy()

In [34]:
df_reviews.shape

(779307, 5)

In [35]:
nltk.download('stopwords')
# Instanciamos el tokenizer
tokenizer=RegexpTokenizer("\w+")

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
stemmer = PorterStemmer()

# Traemos las stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
# Creamos método para aplicar la Normalizacion y luega el Stemming a cada instancia
def processText(text):
    # Vamos a reemplzar los caracteres que no sean letras por espacios
    text=re.sub("[^a-zA-Z]"," ",str(text))
    # Pasamos todo a minúsculas
    text=text.lower()
    # Tokenizamos para separar las palabras del titular
    text=tokenizer.tokenize(text)
    # Eliminamos las palabras de menos de 4 letras
    text = [palabra for palabra in text if len(palabra)>3]
    # Sacamos las Stopwords
    text = [p for p in text if not p in stopwords]
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    # Aplicamos la funcion para buscar la raiz de las palabras
#     text = [stemmer.stem(t) for t in text]
    # Por ultimo volvemos a unir el titular
    text = " ".join(text)
    
    return text

In [37]:
# Aplicamos la transformación
df_reviews['cleaned_text'] = df_reviews['text'].progress_apply(processText)

  0%|          | 0/779307 [00:00<?, ?it/s]

In [38]:
# Mostramos el dataset limpio
df_reviews.loc[np.random.choice(df_reviews.index, size=5)]

,username,hours,product_id,text,user_id,cleaned_text
679151,des,59.4000,10180,good game,424147,good game
424290,Maori,23.6000,249050,One of the most addicting games I've played in...,212643,addicting games played gameplay solid fact sti...
441406,Joseph Stalin,0.8000,434570,"Fun gameplay, gets a bit repetetive after a wh...",173027,gameplay gets repetetive worth buck
116655,DragonElderX9,23.4000,236090,Everything about this game is fantastic! The ...,99867,everything game fantastic gorgeous sound music...
624943,CanneDeBonbon,8.6000,57690,A+\nVery dictator\nSo politics\nMuch Swiss Ban...,58251,dictator politics much swiss bank amaze


In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [41]:
sia = SentimentIntensityAnalyzer()
def polaridad(text):
    return sia.polarity_scores(text)['compound']

In [42]:
df_reviews['polarity'] = df_reviews['text'].progress_apply(polaridad)

  0%|          | 0/779307 [00:00<?, ?it/s]

In [43]:
df_reviews.head()

,username,hours,product_id,text,user_id,cleaned_text,polarity
0,Chaos Syren,0.1000,725280,This would not be acceptable as an entertainme...,62543,would acceptable entertainment even back graph...,0.2354
1,Ariman1,13.2000,328100,Addictive RPG ! Works fine on linux though it ...,26902,addictive works fine linux though minor bugs r...,0.2714
2,freakfantom,0.1000,725280,Прикольная стрелялка. Взял дешево на распродаже.,434604,,0.0000
3,The_Cpt_FROGGY,7.8000,631920,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,347904,somewhere zibylon zibylon command centre read ...,0.9913
4,the_maker988,8.2000,35140,"This game was way to linear for me, and compar...",503851,game linear compared others good dollars winte...,0.3818


In [44]:
# Creamos una columns de Rating entre 0 y 5, escalando la columna 'polarity'
column=df_reviews[['polarity']]
scaler = MinMaxScaler(feature_range=(0,5))
df_reviews['rating'] = scaler.fit_transform(column)
df_reviews

,username,hours,product_id,text,user_id,cleaned_text,polarity,rating
0,Chaos Syren,0.1000,725280,This would not be acceptable as an entertainme...,62543,would acceptable entertainment even back graph...,0.2354,3.0885
1,Ariman1,13.2000,328100,Addictive RPG ! Works fine on linux though it ...,26902,addictive works fine linux though minor bugs r...,0.2714,3.1785
2,freakfantom,0.1000,725280,Прикольная стрелялка. Взял дешево на распродаже.,434604,,0.0000,2.5000
3,The_Cpt_FROGGY,7.8000,631920,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,347904,somewhere zibylon zibylon command centre read ...,0.9913,4.9783
4,the_maker988,8.2000,35140,"This game was way to linear for me, and compar...",503851,game linear compared others good dollars winte...,0.3818,3.4545
...,...,...,...,...,...,...,...,...
779302,Vidaar,783.5000,252490,Summary: Rust is a survival game created by Fa...,366851,summary rust survival game created facepunch s...,-0.9928,0.0180
779303,Nikolai Belinski,55.1000,252490,Coolest Game Eva,242195,coolest game,0.0000,2.5000
779304,RancorZealot,203.5000,252490,Should you buy RUST? Uh... YES. The BEST Survi...,276242,rust best survival sandbox game market playing...,0.7899,4.4748
779305,Jerry,139.8000,252490,Great game!,168748,great game,0.6588,4.1470


In [45]:
# Creo funcion para identificar el rol de cada palabra en el texto del comentario
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [46]:
# Creamos método para aplicar la Normalizacion y luega el Stemming a cada instancia
def processText(text):
    
    # Vamos a reemplzar los caracteres que no sean letras por espacios
    text = re.sub("[^a-zA-Z]"," ",str(text))
    
    # Pasamos todo a minúsculas
    text = text.lower()
    
    # Tokenizamos para separar las palabras del titular
    text = tokenizer.tokenize(text)
    
    # Eliminamos las palabras de menos de 4 letras
    text = [palabra for palabra in text if len(palabra) > 3]
    
    # Sacamos las Stopwords
    text = [p for p in text if not p in stopwords]
    
    # Aplicamos el Lemmatizer (Esto puede tardar un ratito)
#     text = [wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text]
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    # Aplicamos la funcion para buscar la raiz de las palabras
#     text = [stemmer.stem(t) for t in text]
    
    # Por ultimo volvemos a unir el titular
    text = " ".join(text)
    
    return text

In [47]:
# Aplicamos la transformación
if PROCESS_TEXT:
    df_reviews['cleaned_text'] = df_reviews['text'].progress_apply(processText)
    df_reviews.to_csv('df_reviews.csv', index_label='index')
else:
    df_reviews = pd.read_csv('df_reviews.csv')
    df_reviews.drop(columns='index', inplace=True)

  0%|          | 0/779307 [00:00<?, ?it/s]

In [48]:
# Mostramos el dataset limpio
rand_list = np.random.choice(df_reviews.index, size=5)
df_reviews.loc[rand_list]

,username,hours,product_id,text,user_id,cleaned_text,polarity,rating
673480,doomsayer1,0.4000,635320,"I have only played for 10 minutes, but the gam...",426272,played minutes game beautiful well done people...,0.9918,4.9795
297588,DarkWolf,64.8000,301520,Having played this game in the past ( past yea...,84455,played game past past year going give unbiased...,-0.9707,0.0732
59865,lesni.vila,4.6000,391720,"the first hour or so was scary and fun, the re...",459905,first hour scary rest stupid boring repeating ...,-0.7076,0.7310
287270,SoupsUp,48.8000,294100,thank you i think this is mostly enjoyable,318020,thank think mostly enjoyable,0.6597,4.1493
338418,semsemtheman,1.0000,247730,This Game could have been one of the best surv...,491965,game could best survival games devs take money...,-0.5719,1.0703


In [49]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [50]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
def polaridad(text):
    return sia.polarity_scores(text)['compound']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\elmai\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [51]:
df_reviews.isna().sum()

username           0
hours           2637
product_id         0
text               0
user_id            0
cleaned_text       0
polarity           0
rating             0
dtype: int64

In [ ]:
if POLARITY:
    df_reviews['polarity'] = df_reviews['cleaned_text'].progress_apply(polaridad)
    df_reviews.to_csv('df_reviews.csv', index_label='index')
else:
    df_reviews = pd.read_csv('df_reviews.csv')
    df_reviews.drop(columns='index', inplace=True)

  0%|          | 0/779307 [00:00<?, ?it/s]

In [ ]:
# Creamos una columns de Rating entre 0 y 5, escalando la columna 'polarity'
column=df_reviews[['polarity']]
scaler = MinMaxScaler(feature_range=(0,5))
df_reviews['rating'] = scaler.fit_transform(column)
# df_reviews['rating'] = round(df_reviews['rating'] * 4) / 4
df_reviews.loc[rand_list]

In [ ]:
reviews_count = df_reviews.groupby('user_id').count().text.sort_values(ascending=False).reset_index().rename(columns={'text':'count'})
# reviews_count = df_reviews.groupby('user_id').count().text.sort_values(ascending=False).reset_index().rename(columns={'text':'count'})
reviews_count.head()

In [ ]:
df_ratings = df_reviews.groupby('product_id').agg({'rating':['count','mean']}).reset_index()
df_ratings.columns = ['product_id', 'count', 'mean']
df_ratings.sort_values(by='mean', ascending=False, inplace=True)
df_ratings.reset_index(inplace=True, drop=True)
df_ratings

In [ ]:
df_filter = df_ratings[df_ratings['count'] >= 1]

plt.figure(figsize=(7,2))
sns.histplot(data=df_filter, x='mean')
plt.ylabel('count')
plt.xlabel('rating')
plt.xticks(rotation=90)
plt.show()

x = df_filter.index
y1 = df_filter['count']
y2 = df_filter['mean']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('game')
ax1.set_ylabel('count', color='g')
ax2.set_ylabel('rating', color='b')

plt.show()

In [ ]:
df_reviews.shape

In [ ]:
df_reviews.cleaned_text = df_reviews.cleaned_text.str.strip()
df_reviews.cleaned_text.replace('', np.nan, inplace=True)

In [ ]:
df_reviews.isna().sum()

In [ ]:
# df_reviews.dropna(subset=['cleaned_text'], inplace=True)

In [ ]:
df_rep = df_reviews.groupby('cleaned_text').agg({'cleaned_text':'count','rating':'mean'})
df_rep[df_rep.cleaned_text > 3]
# df_reviews[df_reviews.cleaned_text == 'addictive']

## Filtro Colaborativo a partir de descomposición UV

In [ ]:
df_reviews.loc[rand_list]

#### Dataset y Train/Test Split

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df_reviews[['user_id', 'product_id', 'rating']], reader)
trainset, testset = train_test_split(data, random_state=42, test_size=.2)

#### Entrenamiento

In [ ]:
from surprise import SVD, SVDpp, SlopeOne, BaselineOnly
algo = SVD()

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.mae(predictions)
accuracy.rmse(predictions)

#### Optimizacion del Modelo

##### Optimizacion con Grid Search

In [ ]:
stop

In [ ]:
param_grid = {'n_factors': [5,50,100,150],
              'n_epochs': [5,10,20],
              'lr_all': [0.001,0.002,0.005],
              'reg_all': [0.002,0.02,0.2]}

gs = RandomizedSearchCV(SVD,
                  param_grid,
                  measures=['rmse'],
                  cv=3,
                  n_jobs = -1,
                  joblib_verbose =10)

gs.fit(data)

In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

##### Optimizacion manual

In [ ]:
from surprise.model_selection import cross_validate

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128]

for factor in tqdm(factores):
    print(f'\nNúmero de Factores: {factor}')
    algo = SVD(n_factors=factor)
    cv = cross_validate(algo, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

In [ ]:
plt.scatter(factores, rmse_test_means)
plt.xlabel('Numero de factores')
plt.ylabel('Error RMSE')
plt.show()

### Optmizacion del Rating

Una forma de mejorar la performance del modelo seria mejorar la calidad de los datos.
Debido a que contamos con la informacion del tiempo dedicado a cada juego por parte de cada jugador, podriamos inferir que cuanto mas tiempo le ha didcado el usuario al juego en cuestion, mas interes y por ende mayor preferencia por el juego representa.
Por todo esto, podriamos cuantificar las horas dedicadas a jugar dentro del rating: a mayor cantidad de tiempo, se le agregara un plus al rating calculado en base al comentario.

In [ ]:
df_reviews.head()

In [ ]:
# Creamos un dataframe con las cantidad de horas minima y maxima dedicada a cada juego.
df_hours = df_reviews.groupby('product_id').hours.agg(minimo=np.min, maximo=np.max).sort_values(by='maximo', ascending=False)
df_hours

In [ ]:
# Vemos que hay missings. Vamos a ver cuantos hay.
df_hours.isna().sum()

In [ ]:
# Rellenamos los minimos con 0 y los maximos con 1
df_hours.minimo.fillna(0, inplace=True)
df_hours.maximo.fillna(1, inplace=True)
df_hours

In [ ]:
def topMax(index):
    top = df_reviews[df_reviews.product_id==index].quantile(.95).hours
    if np.isnan(top):
        top = 1
    return top

In [ ]:
df_hours.reset_index(inplace=True)

In [ ]:
df_hours['top'] = df_hours.product_id.progress_apply(topMax)
df_hours.sort_values(by='top', ascending=False)

In [ ]:
df_2 = pd.merge(df_reviews, df_hours, left_on='product_id', right_on='product_id')
df_2.shape

In [ ]:
df_2['h'] = np.minimum((df_2['hours']-df_2['minimo'])/(df_2['top']-df_2['minimo'])*2-1, 1)
df_2['h'] = df_2['h'].fillna(0)
df_2

In [ ]:
df_2.describe()

In [ ]:
plt.figure(figsize=(7,2))
sns.histplot(data=df_2, x='h')
plt.ylabel('count')
plt.xlabel('hours')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_2['h_rating'] = df_2['rating'] + df_2['h']

plt.figure(figsize=(7,2))
sns.histplot(data=df_2, x='h_rating')
plt.ylabel('count')
plt.xlabel('hours')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_2['h_rating'] = scaler.fit_transform(df_2[['h_rating']])

plt.figure(figsize=(7,2))
sns.histplot(data=df_2, x='h_rating')
plt.ylabel('count')
plt.xlabel('hours')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_ratings = df_2.groupby('product_id').agg({'h_rating':['count','mean']}).reset_index()
df_ratings.columns = ['product_id', 'count', 'mean']
df_ratings.sort_values(by='mean', ascending=False, inplace=True)
df_ratings.reset_index(inplace=True, drop=True)
df_ratings

In [ ]:
df_filter = df_ratings[df_ratings['count'] >= 1]

plt.figure(figsize=(7,2))
sns.histplot(data=df_filter, x='mean')
plt.ylabel('count')
plt.xlabel('rating')
plt.xticks(rotation=90)
plt.show()

#### Dataset y Train/Test Split

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df_2[['user_id', 'product_id', 'h_rating']], reader)
trainset, testset = train_test_split(data, random_state=42, test_size=.2)

#### Entrenamiento

In [ ]:
from surprise import SVD, SVDpp, SlopeOne, BaselineOnly
algo = SVD()

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.mae(predictions)
accuracy.rmse(predictions)

In [ ]:
# Exploremos las densidaes de distribucion de horas de los juegos con mas horas

for product_id in df_hours[:5].product_id:    
    plt.figure(figsize=(7,2))
    sns.histplot(data=df_reviews[df_reviews.product_id==product_id], x='hours')
    plt.ylabel('count')
    plt.xlabel('hours')
    plt.xlim(0,df_hours[df_hours.product_id==product_id].top.values[0])
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for product_id in df_hours[1500:1505].product_id:    
    plt.figure(figsize=(7,2))
    sns.histplot(data=df_reviews[df_reviews.product_id==product_id], x='hours')
    plt.ylabel('count')
    plt.xlabel('hours')
    plt.xlim(0,df_hours[df_hours.product_id==product_id].top.values[0])
    plt.xticks(rotation=90)
    plt.show()